# AAMR - Custom Model Training - Part 2

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10.13

## Overview

This notebook is used demonstrate custom model training capability using Tensorflow Recommenders.

### Objective

1. Load custom data into tensorflow dataset
2. Create a custom model 
3. Validate custom model recommendations
4. Save model to gcs location for deployment and serving

### Dataset

1. Medications master set
2. Patient Current medications list

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 


In [26]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [27]:
%%capture
!pip install tensorflow
!pip install -q scann
!pip install tensorflow-recommenders

In [28]:
#!pip show tensorflow

In [29]:
#! pip3 install --upgrade --quiet google-cloud-aiplatform

### Import libraries

In [30]:
%%capture
import os
import pprint
import tempfile

from typing import Dict, Text

import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [31]:
import tensorflow_recommenders as tfrs

In [32]:
#from google.cloud import aiplatform

## Preparing the dataset

Let's first have a look at the data.

We will load the Custom data set from csv files into [Tensorflow Datasets](https://www.tensorflow.org/datasets).

In [33]:
med_data = pd.read_csv(
    "data/Medicine_Details.csv",
    names=["Medicine_Name","Composition","Uses","Side_effects","Image_URL","Manufacturer","Excellent_Review_Percentage","Average_Review_Percentage","Poor_Review_Percentage"],
    dtype = str)

#med_data.head()

In [34]:
patient_health_data = pd.read_csv(
    "data/patients_health_mapped_data.csv",
    names=["Diabetic","AlcoholLevel","HeartRate","BloodOxygenLevel","BodyTemperature","Weight","MRI_Delay","Medicine_Name","Dosage_in_mg","Age","Education_Level","Dominant_Hand","Gender","Family_History","Smoking_Status","APOE_E4","Physical_Activity","Depression_Status","Cognitive_Test_Scores","Medication_History","Nutrition_Diet","Sleep_Quality","Chronic_Health_Conditions","Dementia","Patient_Id"],
    dtype = str)

#patient_health_data.head()

In [35]:
%%capture
# All available Medication data
med_dataset = tf.data.Dataset.from_tensor_slices(dict(med_data))

# Patient associated medication data
patient_dataset = tf.data.Dataset.from_tensor_slices(dict(patient_health_data))

In [36]:
# Ratings data.
#ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
#movies = tfds.load("movielens/100k-movies", split="train")

In [37]:
for x in med_dataset.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'Average_Review_Percentage': b'56',
 'Composition': b'Bevacizumab (400mg)',
 'Excellent_Review_Percentage': b'22',
 'Image_URL': b'https://onemg.gumlet.io/l_watermark_346,w_480,h_480/a_ignore'
              b',w_480,h_480,c_fit,q_auto,f_auto/f5a26c491e4d48199ab116a69a9'
              b'69be3.jpg',
 'Manufacturer': b'Roche Products India Pvt Ltd',
 'Medicine_Name': b'Avastin 400mg Injection',
 'Poor_Review_Percentage': b'22',
 'Side_effects': b'Rectal bleeding Taste change Headache Nosebleeds Back pain D'
                 b'ry skin High blood pressure Protein in urine Inflammation of'
                 b' the nose',
 'Uses': b' Cancer of colon and rectum Non-small cell lung cancer Kidney cancer'
         b' Brain tumor Ovarian cancer Cervical cancer'}


In [38]:
for x in patient_dataset.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'APOE_E4': b'Negative',
 'Age': b'60',
 'AlcoholLevel': b'0.08497362913',
 'BloodOxygenLevel': b'96.23074296',
 'BodyTemperature': b'36.22485168',
 'Chronic_Health_Conditions': b'Diabetes',
 'Cognitive_Test_Scores': b'10',
 'Dementia': b'0',
 'Depression_Status': b'No',
 'Diabetic': b'1',
 'Dominant_Hand': b'Left',
 'Dosage_in_mg': b'12',
 'Education_Level': b'Primary School',
 'Family_History': b'No',
 'Gender': b'Female',
 'HeartRate': b'98',
 'MRI_Delay': b'36.42102798',
 'Medication_History': b'No',
 'Medicine_Name': b'Acnesol Gel',
 'Nutrition_Diet': b'Low-Carb Diet',
 'Patient_Id': b'10001',
 'Physical_Activity': b'Sedentary',
 'Sleep_Quality': b'Poor',
 'Smoking_Status': b'Current Smoker',
 'Weight': b'57.56397754'}


In [65]:
patient_dataset_select = patient_dataset.map(lambda x: {
    "Medicine_Name": x["Medicine_Name"],
    "Patient_Id": x["Patient_Id"],
    "Gender" : x["Gender"]
})


In [66]:
med_dataset_select = med_dataset.map(lambda x: x["Medicine_Name"])

In [67]:
tf.random.set_seed(42)
shuffled = patient_dataset_select.shuffle(1000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(800)
test = shuffled.skip(200).take(200)


In [68]:
len(test)

200

In [69]:
med_titles = med_dataset_setlect.batch(1_000)
patient_dataset_ids = patient_dataset_select.batch(1_000).map(lambda x: x["Patient_Id"])

unique_med_titles = np.unique(np.concatenate(list(med_titles)))
unique_patient_dataset_ids = np.unique(np.concatenate(list(patient_dataset_ids)))

unique_med_titles[:10]


array([b'A Doxid 100mg Capsule', b'A Ret HC Cream', b'A-CN Gel',
       b'A-Ret 0.025% Gel', b'A-Ret 0.05% Gel', b'A-Ret 0.1% Gel',
       b'A-Ret 0.5% Cream', b'AA 5 Tablet', b'AB Phylline Capsule',
       b'AB Phylline N Tablet'], dtype=object)

In [70]:
unique_patient_dataset_ids[:10]

array([b'10001', b'10002', b'10003', b'10004', b'10005', b'10006',
       b'10007', b'10008', b'10009', b'10010'], dtype=object)

In [71]:
len(unique_patient_dataset_ids)

1000

## Implementing a model

Choosing the architecture of our model is a key part of modelling.

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

### The query tower

Let's start with the query tower.

The first step is to decide on the dimensionality of the query and candidate representations:

In [72]:
embedding_dimension = 32

In [47]:
user_med_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_patient_dataset_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_patient_dataset_ids) + 1, embedding_dimension)
])

### The candidate tower

We can do the same with the candidate tower.

In [48]:
med_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_med_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_med_titles) + 1, embedding_dimension)
])

### Metrics

In our training data we have positive (user, medication) pairs. To figure out how good our model is, we need to compare the affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.

To do this, we can use the `tfrs.metrics.FactorizedTopK` metric. The metric has one required argument: the dataset of candidates that are used as implicit negatives for evaluation.

In our case, that's the `meds` dataset, converted into embeddings via our movie model:

In [49]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=med_dataset_select.batch(128).map(med_model)
)

### Loss

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the `Retrieval` task object: a convenience wrapper that bundles together the loss function and metric computation:

In [50]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

### The full model

We can now put it all together into a model. 
TFRS exposes a base model class (`tfrs.models.Model`) which streamlines building models: all we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [51]:
class MedRecomModel(tfrs.Model):

  def __init__(self, user_med_model, med_model):
    super().__init__()
    self.med_model: tf.keras.Model = med_model
    self.user_med_model: tf.keras.Model = user_med_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_med_embeddings = self.user_med_model(features["Patient_Id"])
    # And pick out the med features and pass them into the med model,
    # getting embeddings back.
    positive_med_embeddings = self.med_model(features["Medicine_Name"])

    # The task computes the loss and the metrics.
    return self.task(user_med_embeddings, positive_med_embeddings)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [52]:
model = MedRecomModel(user_med_model, med_model)
#Pick righ optimizer for training
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [53]:
cached_train = train.shuffle(1000).batch(10).cache()
cached_test = test.batch(100).cache()

## Train the  model

In [54]:
import datetime
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [55]:
rm -rf ./logs/

In [56]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [57]:
model.fit(cached_train, epochs=3,callbacks=[tensorboard_callback])
%tensorboard --logdir logs/fit

Epoch 1/3
80/80 [==============================] - 16s 174ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0538 - factorized_top_k/top_5_categorical_accuracy: 0.0712 - factorized_top_k/top_10_categorical_accuracy: 0.0875 - factorized_top_k/top_50_categorical_accuracy: 0.1112 - factorized_top_k/top_100_categorical_accuracy: 0.1375 - loss: 23.0244 - regularization_loss: 0.0000e+00 - total_loss: 23.0244
Epoch 2/3
80/80 [==============================] - 13s 168ms/step - factorized_top_k/top_1_categorical_accuracy: 0.4663 - factorized_top_k/top_5_categorical_accuracy: 0.5462 - factorized_top_k/top_10_categorical_accuracy: 0.5663 - factorized_top_k/top_50_categorical_accuracy: 0.6212 - factorized_top_k/top_100_categorical_accuracy: 0.6463 - loss: 22.7042 - regularization_loss: 0.0000e+00 - total_loss: 22.7042
Epoch 3/3
80/80 [==============================] - 13s 166ms/step - factorized_top_k/top_1_categorical_accuracy: 0.7000 - factorized_top_k/top_5_categorical_accuracy: 0.7713 - 

Reusing TensorBoard on port 6006 (pid 86848), started 20:44:34 ago. (Use '!kill 86848' to kill it.)

In [58]:
#cached_test

Finally, we can evaluate our model on the test set:

In [59]:
model.evaluate(cached_test, return_dict=True)

2/2 [==============================] - 1s 270ms/step - factorized_top_k/top_1_categorical_accuracy: 0.7850 - factorized_top_k/top_5_categorical_accuracy: 0.8450 - factorized_top_k/top_10_categorical_accuracy: 0.8750 - factorized_top_k/top_50_categorical_accuracy: 0.9200 - factorized_top_k/top_100_categorical_accuracy: 0.9400 - loss: 415.6057 - regularization_loss: 0.0000e+00 - total_loss: 415.6057


{'factorized_top_k/top_1_categorical_accuracy': 0.7850000262260437,
 'factorized_top_k/top_5_categorical_accuracy': 0.8450000286102295,
 'factorized_top_k/top_10_categorical_accuracy': 0.875,
 'factorized_top_k/top_50_categorical_accuracy': 0.9200000166893005,
 'factorized_top_k/top_100_categorical_accuracy': 0.9399999976158142,
 'loss': 414.11175537109375,
 'regularization_loss': 0,
 'total_loss': 414.11175537109375}

## Making predictions

Now that we have a model, we would like to be able to make predictions. We can use the `tfrs.layers.factorized_top_k.BruteForce` layer to do this.

In [61]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_med_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((med_dataset_select.batch(100), med_dataset_select.batch(100).map(model.med_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["10025"]))
print(f"Recommendations for user 10025: {titles[0, :10]}")

Recommendations for user 10025: [b'Myket Cream' b'Maskofung-XL Cream' b'Medrol 4mg Tablet'
 b'Forfora-Z Shampoo' b'Oxerute Cream' b'Omnitan H Tablet'
 b'Eptoin 300 ER Tablet' b'Dresin Mouthwash &amp; Gargle' b'Ondem Syrup'
 b'Monoguard-B Cream']


## Model serving

After the model is trained, we need a way to deploy it.

In a two-tower retrieval model, serving has two components:

- a serving query model, taking in features of the query and transforming them into a query embedding, and
- a serving candidate model. This most often takes the form of an approximate nearest neighbours (ANN) index which allows fast approximate lookup of candidates in response to a query produced by the query model.

In [63]:
import os
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(index, path)

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["10042"])

  print(f"Recommendations: {titles[0][:3]}")
  dst_path = "/tmp/vc-model-med-select-recomm-2"
  os.rename(path, dst_path)

INFO:tensorflow:Assets written to: /var/tmp/tmp4l5w4yqn/model/assets


Recommendations: [b'Olmetor-H Tablet' b'Ramistar 2.5 Tablet' b'Quel SR 200 Tablet']


In [64]:
! gsutil cp -r "/tmp/vc-model-med-select-recomm-2" "gs://vc-model-training"

Copying file:///tmp/vc-model-med-select-recomm-2/saved_model.pb [Content-Type=application/octet-stream]...
Copying file:///tmp/vc-model-med-select-recomm-2/fingerprint.pb [Content-Type=application/octet-stream]...
Copying file:///tmp/vc-model-med-select-recomm-2/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file:///tmp/vc-model-med-select-recomm-2/variables/variables.index [Content-Type=application/octet-stream]...
- [4 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 4 objects/1.4 MiB.                                      


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:


In [ ]:
import os

# Delete endpoint resource
# e.g. `endpoint.delete()`

# Delete model resource
# e.g. `model.delete()`

# Delete Cloud Storage objects that were created
delete_bucket = True
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI